# SQL with Python

## Programming and Data Management (EDI 3400)

### *Vegard H. Larsen (Department of Data Science and Analytics)*

# 1.  Intro to `sqlite3`

SQLite is a lightweight, serverless, and self-contained relational database system, and `sqlite3` is its accompanying module in the Python Standard Library, enabling seamless interaction with SQLite databases. With `sqlite3`, we can create, connect to, and query SQLite databases directly from Python. Unlike more complex database systems, SQLite doesn't require setup or administration, making it an ideal starting point to grasp fundamental database operations and concepts.

In [1]:
# sqlite3 is part of the Standard Library

import sqlite3

In [2]:
# We can connect to our database
# Make sure the database is stored in the same folder as the notebook

con = sqlite3.connect("auto_dealership_database.db")

In [3]:
# In order to execute SQL statements and fetch results from SQL queries, 
# we will need to use a database cursor.

cur = con.cursor()

## Executing a query

In [4]:
# Now we are connected to the database and we have a cursor 
# and we can execute an SQL query

res = cur.execute("SELECT * FROM Employees")

In [ ]:
# This gives us a cursor object

#res.fetchone()

In [5]:
# We can get all the information from the query using fetchall

employees_table = res.fetchall()

In [ ]:
# The data is then stored as a list of tuples

employees_table

## Convert the list into a Pandas dataframe

In [7]:
import pandas as pd

Employees = pd.DataFrame(employees_table)

In [8]:
# Let's inspect the dataframe

Employees.head(5)

,0,1,2,3,4
0,1,Erik,114100,Bachelor,Sales
1,2,Sue,116200,Bachelor,Admin
2,3,Linda,67200,High School,Admin
3,4,Anne,75900,Master,Service
4,5,Mary,89100,Bachelor,Service


## Let's set the column names

In [9]:
# Get the column names from the Employees table 

tmp = cur.execute("SELECT name FROM PRAGMA_TABLE_INFO('Employees')").fetchall() #tmp is a list of tuples

In [10]:
# Let's inspect the result from the query

tmp

[('id',), ('name',), ('salary',), ('education',), ('department',)]

In [11]:
# Change the list of tuples into a list of strings

employees_column_names = [name[0] for name in tmp]
employees_column_names

['id', 'name', 'salary', 'education', 'department']

In [12]:
# Add the column names to the dataframe

Employees.columns = employees_column_names

In [13]:
# We set the id column to be the index column

Employees.index = Employees['id']
del Employees['id']

In [15]:
# Let's look at the Employees dataframe

Employees.head(10)

,name,salary,education,department
id,,,,
1,Erik,114100,Bachelor,Sales
2,Sue,116200,Bachelor,Admin
3,Linda,67200,High School,Admin
4,Anne,75900,Master,Service
5,Mary,89100,Bachelor,Service
6,Tom,95900,Bachelor,Sales
7,John,148200,Bachelor,Sales
8,Joe,148100,Master,Sales
9,Sofia,117100,Bachelor,Sales


In [14]:
# Remember to close the connection

con.close()

# 2. SQL queries with Pandas  

In [15]:
import sqlite3
import pandas as pd

con = sqlite3.connect("auto_dealership_database.db")
Customers = pd.read_sql("SELECT * FROM Customers", con)
con.close()

In [ ]:
Customers

## The query is defined within a Python string 

In [19]:
query = """
        SELECT *
        FROM Customers
        WHERE birth_year = 1995
        """
# SELECT * means select all columns

In [ ]:
print(query)

In [20]:
con = sqlite3.connect("auto_dealership_database.db")
result = pd.read_sql(query, con)
con.close()

In [ ]:
result

# 3. Let us create a database

### Datatypes in SQLite

- **NULL**. The value is a NULL value.

- **INTEGER**. The value is a signed integer, stored in 0, 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.

- **REAL**. The value is a floating point value, stored as an 8-byte IEEE floating point number.

- **TEXT**. The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).

- **BLOB**. The value is a blob of data, stored exactly as it was input.

In [23]:
## Imports

import pandas as pd
import sqlite3

In [24]:
!rm example_database.db
!ls

'rm' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
con = sqlite3.connect('example_database.db') 

In [ ]:
ls

In [29]:
## Create the cursor

cursor = con.cursor()

In [ ]:
cursor.execute?

In [ ]:
cursor.execute('''CREATE TABLE Employees(id INTEGER PRIMARY KEY,
                                         name TEXT,
                                         salary INTEGER,  
                                         education TEXT)''')

In [ ]:
sqlite_insert = """
                INSERT INTO Employees(id, name, salary, education) 
                VALUES(1, 'Nora', 100000, 'Master')
                """ 

cursor.execute(sqlite_insert)

In [33]:
# Commit the employees data to the database

con.commit()
con.close()

In [34]:
# Let's look at our newly created table

con = sqlite3.connect("example_database.db")
Employees2 = pd.read_sql("SELECT * FROM Employees", con)
con.close()

Employees2

,id,name,salary,education
0,1,Nora,100000,Master


In [ ]:
# We can add another row

con = sqlite3.connect('example_database.db') 
cursor = con.cursor()
sqlite_insert = """
                INSERT INTO Employees(id, name, salary, education) 
                VALUES(2, 'Ted', 90000, 'High school')
                """ 

cursor.execute(sqlite_insert)
cursor.commit()
cursor.close()

In [35]:
con = sqlite3.connect("example_database.db")
Employees2 = pd.read_sql("SELECT * FROM Employees", con)
con.close()

Employees2

,id,name,salary,education
0,1,Nora,100000,Master


# 4. Adding many rows to a database from Python


In [36]:
# Create some random data

import numpy as np

ids = np.arange(1, 101)
ages = np.random.randint(18, 75, size=100)
names = np.random.choice(['John', 'Jane', 'Mary', 'Bob', 'Tom', 'Jack', 'Karl', 'Sue', 'Sally', 'Ida'],
                         size=100)

In [ ]:
# Create the table in our example database

con = sqlite3.connect('example_database.db')
cursor = con.cursor()
cursor.execute('''CREATE TABLE Customers(id INTEGER PRIMARY KEY,
                                         name STRING,
                                         age INTEGER)''')

In [38]:
# We use a for loop to add the data
# We can use a f-string to change the content of the sql-code for each iteration of the loop

for i in range(100):
    sqlite_insert = f"INSERT INTO Customers(id, name, age) VALUES({ids[i]}, '{names[i]}', {ages[i]})"
    con.execute(sqlite_insert)
con.commit() # Commit the changes to the database
con.close()

In [ ]:
# Let's look at the new table

con = sqlite3.connect("example_database.db")
Customers2 = pd.read_sql("SELECT * FROM Customers", con)
con.close()

Customers2